In [1]:
import numpy as np

In [2]:
import csv

#필드 목록 정의
#비정형 필드
title_arr = []

#분류값을 저장할 배열 (0,1)
ctr_arr = []

#텍스트와 분류값이 저장된 파일 불러오기(학습용 데이터셋)
f = open('c:/data/text/ratings_train.csv', encoding='ms949')
reader = csv.reader(f)
max_length = 0
length = 0
max_idx = 0
count = 0
next(reader, None) #헤더 스킵(첫라인을 건너뜀)
count_train = 0
for line in reader:
    #print(line)
    #레코드 전체를 하는 방법
    #텍스트 필드
    title_arr.append(line[0])
    #결과 필드
    ctr_arr.append(int(line[1]))
    #텍스트의 길이
    length = len(line[0])
    #print(length)
    if max_idx < length:
        max_length = length
        max_idx = count
    count += 1
    count_train += 1
    if count_train >= 10000: break

f.close()

#검증용 데이터셋 파일 불러오기
f2 = open('c:/data/text/ratings_test.csv', 'r', encoding='ms949')
reader = csv.reader(f2)
#헤더 스킵(첫라인을 건너뜀)
next(reader, None)
count_test = 0
for line in reader:
    #print(line)
    #레코드 전체를 하는 방법
    title_arr.append(line[0]) #텍스트 필드
    #결과 필드
    ctr_arr.append(int(line[1]))
    #텍스트의 길이
    length = len(line[0])
    #print(length)
    if max_length < length:
        max_length = length
        max_idx = count
    count += 1
    count_test +=1
    if count_test >= 10000: break

f2.close()

In [3]:
print('count_train:', count_train)
print('count_test:', count_test)
print('title_arr size:', len(title_arr))
print('ctr_arr size:', len(ctr_arr))
print('max_length:', max_length)

count_train: 10000
count_test: 10000
title_arr size: 20000
ctr_arr size: 20000
max_length: 144


In [4]:
from konlpy.tag import Okt
okt = Okt()
#명사, 형용사, 동사, 알파벳, 부사
title_noun_arr = []
for index, title in enumerate(title_arr):
    if index % 1000 == 0:
        print('step:', index)
        #명사만 추출
    title_noun_arr.append(okt.nouns(title))
print(len(title_noun_arr))

step: 0
step: 1000
step: 2000
step: 3000
step: 4000
step: 5000
step: 6000
step: 7000
step: 8000
step: 9000
step: 10000
step: 11000
step: 12000
step: 13000
step: 14000
step: 15000
step: 16000
step: 17000
step: 18000
step: 19000
20000


In [5]:
#벡터모델을 불러와서 텍스트를 벡터로변환
import gensim

MIN_NOUN_VECTOR_VALUE = -10.0
MAX_NOUN_VECTOR_VALUE = 10.0
NOUN_VECTOR_SIZE = 100

#벡터 생성 함수
def generate_random_noun_vector():
    return np.random.uniform(low=MIN_NOUN_VECTOR_VALUE,
    high=MAX_NOUN_VECTOR_VALUE,
    size=(NOUN_VECTOR_SIZE,))

#word2vec 모델 불러오기
w2v_model = gensim.models.Word2Vec.load('c:/data/text/text_100.model')
title_noun_vector_arr = [] #벡터를 저장할 리스트
for index, title_nouns in enumerate(title_noun_arr):
    if index % 10000 == 0:
        print('step:', index)
    noun_vector_arr = []
    for noun in title_nouns:
        try:
            #단어별로 벡터모델 적용
            noun_vector = w2v_model[noun]
        except Exception as e:
            #단어사전에 존재하지 않으면 기본 벡터 생성
            noun_vector = generate_random_noun_vector()
        noun_vector_arr.append(noun_vector)
    #if len(noun_vector_arr) > 0:
    # print(len(noun_vector_arr))
    title_noun_vector_arr.append(noun_vector_arr)        

step: 0
step: 10000


In [6]:
print(title_noun_vector_arr[:1])

[[array([-9.71231325,  3.86557533, -8.06305422,  6.32537441, -3.49034531,
        0.32120118, -5.83691596,  8.93927049, -5.47432932,  8.22410006,
        3.27404084, -0.2011729 ,  4.92367754, -1.15045142,  3.31961015,
        7.35308813,  0.01978571, -6.84022468,  9.12227407, -4.73806968,
       -2.73970862, -2.40041667,  4.62574251,  2.23344741, -1.86565948,
        8.51726143, -4.57703675, -1.92221069,  3.72729252,  3.22010981,
       -4.09860343,  4.81284588, -9.45301183,  9.00395216,  1.21533502,
       -7.17122814, -1.92758592,  7.83598351,  9.32500204,  8.24614769,
       -5.61018458, -6.01151613, -6.77462352,  7.41561149,  5.13859779,
        5.54210275,  3.14762009, -1.53051266, -1.28133636,  4.98691168,
        7.8165923 , -5.73501258, -4.94005832,  7.3191278 ,  6.8335864 ,
        5.26009003, -3.09360885, -4.63462549, -6.91234614, -1.16542007,
       -4.77138727, -3.96439814,  4.59125333, -1.64899187, -2.06088974,
       -3.78711936, -9.54277496,  8.41685783, -6.18463028,  7.

In [7]:
max_size = 0 #가장 긴 텍스트 사이즈를 저장할 변수
min_size = 100
avg = 0
count = 0
sum_size = 0

#가장 긴 리뷰 텍스트 계산
for index, title_noun_vector in enumerate(title_noun_vector_arr):
    #print(len(title_noun_vector))
    if len(title_noun_vector) == 0:
        pass
        #print(title_noun_vector)
    sum_size += len(title_noun_vector)
    if max_size < len(title_noun_vector):
        max_size = len(title_noun_vector)
    if min_size > len(title_noun_vector):
        min_size = len(title_noun_vector)
    count += 1
    #print(':title_noun_vector len=>', len(title_nount_vector))
#padding
TITLE_LENGTH = max_size

def generate_zero_noun_vector():
    return np.random.uniform(low=0.0, high=0.0, size=(NOUN_VECTOR_SIZE,))

#인덱스와 요소를 하나씩 전개하여 패딩 처리
title_noun_vector_arr2 = []
for index, title_noun_vector in enumerate(title_noun_vector_arr):
    if index % 1000 == 0:
        print('step:', index)
    while len(title_noun_vector) < TITLE_LENGTH:
        title_noun_vector.append(generate_zero_noun_vector())
    title_noun_vector = title_noun_vector[:TITLE_LENGTH]
    title_noun_vector_arr2.append(title_noun_vector)

step: 0
step: 1000
step: 2000
step: 3000
step: 4000
step: 5000
step: 6000
step: 7000
step: 8000
step: 9000
step: 10000
step: 11000
step: 12000
step: 13000
step: 14000
step: 15000
step: 16000
step: 17000
step: 18000
step: 19000


In [8]:
print(min_size, max_size)

0 37


In [9]:
#ctr to classification
#클래스 개수
NUM_CLASSES = 2
ctr_class_arr = []
#클래스 개수 저장
ctr_class_count = [0,0]
#print(success_arr)
for index, ctr in enumerate(ctr_arr):
    if ctr ==0:
        #ctr_class_arr.append([1,0])
        ctr_class_arr.append(0.0)
        ctr_class_count[0] += 1
    elif ctr == 1:
        #ctr_class_arr.append([0,1])
        ctr_class_arr.append(1.0)
        ctr_class_count[1] +=1
print(ctr_class_count)
print(ctr_class_arr[:10])

[9960, 10040]
[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]


In [10]:
#데이터 전처리(0.0~1.0), 각 특성의 평균을 계산
def prepare_data(arr):
    sum1 = 0
    for i in arr:
        sum1 += i
    arr = [float(i)/sum1 for i in arr]
    return arr
#전처리가 필요한 필드에 대해 함수 호출

In [11]:
#검증용 데이터셋의 개수 구하기
test_data_size = count_test
train_data_size = count_train
print(train_data_size, ' ', test_data_size)

#학습용 데이터셋과 검증용 데이터셋을 분리하는 코드
train_input = title_noun_vector_arr[0:train_data_size]
train_label = ctr_class_arr[0:train_data_size]
test_input = title_noun_vector_arr[train_data_size:]
test_label = ctr_class_arr[train_data_size:]

print('ctr_class_arr 개수:', len(ctr_class_arr))
print('train 총데이터개수:', len(train_input))
print('train라벨개수:', len(train_label))
print('test총데이터개수', len(test_input))
print('test라벨개수:', len(test_label))
print('train_data_size:', train_data_size)
print('test_data_size:', test_data_size)

10000   10000
ctr_class_arr 개수: 20000
train 총데이터개수: 10000
train라벨개수: 10000
test총데이터개수 10000
test라벨개수: 10000
train_data_size: 10000
test_data_size: 10000


In [12]:
len(train_input)

10000

In [13]:
len(test_input)

10000

In [14]:
#데이터를 케라스에 맞게 변환해주는 과정
#데이터개수, 가로, 세로, 채널수(흑백이미지는 1, 컬러는 3, 텍스트는 1)
#width, height, channel

train_input = np.array(train_input)
train_input = train_input.reshape(train_input.shape[0],
train_input.shape[1], NOUN_VECTOR_SIZE, 1)
test_input = np.array(test_input)
test_input = test_input.reshape(test_input.shape[0],
train_input.shape[1], NOUN_VECTOR_SIZE, 1)

#X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [15]:
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model

#입력데이터
input_a = Input(shape=(train_input.shape[1], NOUN_VECTOR_SIZE, 1),
name='input-layer')

#첫번째 Conv 레이어
x = Conv2D(1, (3,3), activation='relu', padding='valid', strides=(1,1))(input_a)

#두번째 Conv 레이어
x = Conv2D(1, (3,3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.25)(x)

#fully connected network
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

#출력계층(1개의 클래스로 분류됨)
out = Dense(1, activation='sigmoid', name='output-layer')(x)

#CNN모델이 생성됨
model = Model(inputs = [input_a], outputs = out)

#모델 컴파일, 손실함수, 옵티마이저 설정
#CNN 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
#학습
from tensorflow.keras.callbacks import EarlyStopping

#조기종료 콜백함수
early_stopping = EarlyStopping(patience = 2)
hist = model.fit(x=train_input, y=np.array(train_label),
validation_split=0.25, batch_size=256, epochs=5, verbose=1, callbacks=[early_stopping])

Epoch 1/5
30/30 [==============================] - 3s 84ms/step - loss: 0.8285 - accuracy: 0.4942 - val_loss: 0.7003 - val_accuracy: 0.5072
Epoch 2/5
30/30 [==============================] - 2s 61ms/step - loss: 0.7003 - accuracy: 0.5142 - val_loss: 0.6953 - val_accuracy: 0.5060
Epoch 3/5
30/30 [==============================] - 2s 61ms/step - loss: 0.6954 - accuracy: 0.5079 - val_loss: 0.6957 - val_accuracy: 0.4968
Epoch 4/5
30/30 [==============================] - 2s 63ms/step - loss: 0.6875 - accuracy: 0.5217 - val_loss: 0.6968 - val_accuracy: 0.4908


In [17]:
#평가
scores = model.evaluate(np.array(test_input)[:100],
np.array(test_label)[:100], batch_size=1, verbose=1)

#손실률
print('%s: %.2f%%' % (model.metrics_names[0], scores[0] * 100))

#정확도
print('%s:%2f%%' % (model.metrics_names[1], scores[1]*100))

#CNN의 최종 출력값이 필요한 경우 아래 코드 사용
m2 = Model(inputs = model.input, outputs = model.get_layer('output-layer').output)
y = m2.predict(test_input)
#y = m2.predict(train_input)

100/100 [==============================] - 0s 1000us/step - loss: 0.6942 - accuracy: 0.5100
loss: 69.42%
accuracy:50.999999%


In [18]:
y[0:10]

array([[0.5008367 ],
       [0.5008367 ],
       [0.5003541 ],
       [0.47448525],
       [0.49507254],
       [0.44151315],
       [0.50771487],
       [0.5012644 ],
       [0.40585366],
       [0.4645676 ]], dtype=float32)

In [19]:
model.save('c:/data/text')

INFO:tensorflow:Assets written to: c:/data/text\assets


In [20]:
n = 911
print(model.predict(np.array(train_input)[n].reshape(1,37,100,1)))
print(np.array(train_label)[n])
print(title_arr[n])

[[0.5008367]]
1.0
재밌군


In [21]:
n = 5000
print(model.predict(np.array(test_input)[n].reshape(1,37,100,1)))
print(np.array(test_label)[n])
print(title_arr[n+10000])

[[0.47915876]]
0.0
OOO기 영화입니다. 긴장감도 그렇게 없고 반전도 안놀랍고 시간이 아까운 영화입니다.
